# Relative Strength Index (RSI) - Complete Trading Tutorial

## Overview
This notebook provides a comprehensive tutorial on the Relative Strength Index (RSI) indicator, including:
- Concept and theory
- Mathematical formula and calculation
- Manual implementation in Python
- TradingView integration
- Binance platform usage
- Practical trading strategies
- Backtesting examples

## 1. Introduction to Relative Strength Index (RSI)

### What is RSI?
The Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements. Developed by J. Welles Wilder Jr. in 1978, it oscillates between 0 and 100 and is typically used to identify overbought and oversold conditions.

### Why Use RSI?
- **Overbought/Oversold**: Identifies extreme price levels
- **Divergence**: Detects potential trend reversals
- **Momentum**: Measures the strength of price movements
- **Trend Confirmation**: Confirms the strength of existing trends

### Best Timeframes and Markets
- **Short-term**: RSI(9-14) for day trading
- **Medium-term**: RSI(14-21) for swing trading
- **Long-term**: RSI(21-30) for position trading
- **Markets**: Works well in both trending and ranging markets

## 2. Mathematical Formula

### RSI Formula
```
RSI = 100 - [100 / (1 + RS)]

Where:
- RS = Average Gain / Average Loss
- Average Gain = Sum of gains over n periods / n
- Average Loss = Sum of losses over n periods / n
```

### Step-by-Step Calculation:
1. Calculate price changes (Δ = Price[t] - Price[t-1])
2. Separate gains (positive changes) and losses (negative changes)
3. Calculate average gain and average loss over n periods
4. Calculate RS = Average Gain / Average Loss
5. Calculate RSI = 100 - [100 / (1 + RS)]

### Parameters
- **Period (n)**: Number of time periods (default: 14)
- **Overbought**: Typically 70 or 80
- **Oversold**: Typically 30 or 20

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import sys
sys.path.append('../utils')

# Import utilities
from data_downloader import DataDownloader
from indicators import IndicatorCalculator
from backtest_engine import BacktestEngine, StrategyGenerator

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

## 3. Data Download

Let's download historical crypto data for our analysis.

In [ ]:
# Initialize data downloader
downloader = DataDownloader()

# Download BTC/USDT data
pair = "BTC/USDT"
timeframe = "1d"
days = 365

print(f"Downloading {pair} data for {days} days...")
df = downloader.get_market_data(pair, timeframe, days, source='ccxt')

if df is not None:
    print(f"\nDownloaded {len(df)} candles of {pair} data")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    print(f"\nFirst few rows:")
    display(df.head())
else:
    print("Failed to download data")

## 4. Manual Calculation of RSI

Let's implement RSI from scratch to understand how it works.

In [ ]:
def calculate_rsi_manual(data, period=14):
    """
    Calculate RSI manually
    """
    delta = data.diff()
    
    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Calculate RS and RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate RSI manually
df_manual = df.copy()
df_manual['rsi_14'] = calculate_rsi_manual(df_manual['close'], 14)
df_manual['rsi_21'] = calculate_rsi_manual(df_manual['close'], 21)

print("RSI calculated manually:")
display(df_manual[['close', 'rsi_14', 'rsi_21']].tail(10))

# Let's verify the calculation manually
print("\nManual verification of RSI(14) for the last row:")
recent_data = df_manual['close'].tail(15)  # 14 periods + 1 for calculation
delta = recent_data.diff().dropna()
gain = delta[delta > 0]
loss = -delta[delta < 0]
avg_gain = gain.mean()
avg_loss = loss.mean()
rs = avg_gain / avg_loss
manual_rsi = 100 - (100 / (1 + rs))
calculated_rsi = df_manual['rsi_14'].iloc[-1]

print(f"Average gain: {avg_gain:.4f}")
print(f"Average loss: {avg_loss:.4f}")
print(f"RS: {rs:.4f}")
print(f"Manual RSI(14): {manual_rsi:.2f}")
print(f"Calculated RSI(14): {calculated_rsi:.2f}")
print(f"Difference: {abs(manual_rsi - calculated_rsi):.6f}")

## 5. Library Implementation

Now let's use established libraries to calculate RSI.

In [ ]:
# Using our utilities
df_utils = df.copy()
df_utils['rsi_14_utils'] = IndicatorCalculator.rsi(df_utils['close'], 14)

# Using pandas-ta
import pandas_ta as ta
df_ta = df.copy()
df_ta.ta.rsi(length=14, append=True)

# Using TA-Lib
try:
    import talib
    df_talib = df.copy()
    df_talib['rsi_14_talib'] = talib.RSI(df_talib['close'].values, timeperiod=14)
    print("TA-Lib RSI calculated successfully")
except ImportError:
    print("TA-Lib not installed, skipping TA-Lib example")
    df_talib = df.copy()
    df_talib['rsi_14_talib'] = np.nan

# Compare results
print("Comparison of different RSI implementations:")
comparison = pd.DataFrame({
    'Manual': df_manual['rsi_14'],
    'Utils': df_utils['rsi_14_utils'],
    'Pandas-TA': df_ta['RSI_14'],
    'TA-Lib': df_talib['rsi_14_talib']
})

display(comparison.dropna().tail(10))

# Check if all methods produce similar results
print("\nDifferences between methods:")
print(f"Manual vs Utils max difference: {abs(comparison['Manual'] - comparison['Utils']).max():.8f}")
print(f"Manual vs Pandas-TA max difference: {abs(comparison['Manual'] - comparison['Pandas-TA']).max():.8f}")

## 6. Visualization

Let's visualize RSI with price action.

In [ ]:
# Create interactive plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'{pair} - Price Action', f'{pair} - RSI'),
    vertical_spacing=0.1,
    row_heights=[0.6, 0.4]
)

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_manual.index,
        open=df_manual['open'],
        high=df_manual['high'],
        low=df_manual['low'],
        close=df_manual['close'],
        name='Price'
    ),
    row=1, col=1
)

# RSI line
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['rsi_14'],
        mode='lines',
        name='RSI(14)',
        line=dict(color='purple', width=2)
    ),
    row=2, col=1
)

# Overbought/Oversold levels
fig.add_hline(y=70, line=dict(color='red', dash='dash'), annotation_text='Overbought (70)', row=2, col=1)
fig.add_hline(y=30, line=dict(color='green', dash='dash'), annotation_text='Oversold (30)', row=2, col=1)
fig.add_hline(y=50, line=dict(color='gray', dash='dot'), annotation_text='Midline (50)', row=2, col=1)

# Highlight overbought/oversold areas
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['rsi_14'],
        mode='markers',
        marker=dict(
            size=4,
            color=np.where(df_manual['rsi_14'] > 70, 'red', 
                         np.where(df_manual['rsi_14'] < 30, 'green', 'purple'))
        ),
        name='RSI Levels',
        showlegend=False
    ),
    row=2, col=1
)

fig.update_layout(
    title=f'{pair} - Relative Strength Index (RSI)',
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False
)

fig.show()

# Create static plot showing RSI characteristics
plt.figure(figsize=(15, 10))

# Price and RSI
plt.subplot(2, 1, 1)
plt.plot(df_manual.index, df_manual['close'], label='Price', alpha=0.7, color='black')
plt.title(f'{pair} - Price Action')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_manual.index, df_manual['rsi_14'], label='RSI(14)', alpha=0.8, color='purple')
plt.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold (30)')
plt.axhline(y=50, color='gray', linestyle=':', alpha=0.5, label='Midline (50)')

# Color background based on RSI levels
plt.fill_between(df_manual.index, 30, 70, alpha=0.1, color='gray', label='Neutral Zone')
plt.fill_between(df_manual.index, 70, 100, alpha=0.1, color='red', label='Overbought Zone')
plt.fill_between(df_manual.index, 0, 30, alpha=0.1, color='green', label='Oversold Zone')

plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

## 7. TradingView Integration

### Pine Script for RSI

```pine
// Relative Strength Index (RSI) Pine Script
//@version=5
indicator("Relative Strength Index", shorttitle="RSI", format=format.price, precision=2)

// Input parameters
len = input.int(14, title="Length", minval=1)
src = input.source(close, "Source")
overbought = input.int(70, "Overbought", minval=50, maxval=100)
oversold = input.int(30, "Oversold", minval=0, maxval=50)

// Calculate RSI
up = ta.rma(math.max(src - src[1], 0), len)
down = ta.rma(math.max(src[1] - src, 0), len)
rsi = down == 0 ? 100 : up == 0 ? 0 : 100 - (100 / (1 + up / down))

// Plot RSI
plot(rsi, "RSI", color=color.new(color.purple, 0))
band1 = hline(overbought, "Overbought", color=color.new(color.red, 50))
band0 = hline(oversold, "Oversold", color=color.new(color.green, 50))
fill(band1, band0, color=color.new(color.gray, 90), title="Middle Band")

// Add signals
buySignal = ta.crossover(rsi, oversold)
sellSignal = ta.crossunder(rsi, overbought)

plotshape(buySignal, title="Buy Signal", location=location.belowbar, color=color.green, style=shape.triangleup, size=size.small)
plotshape(sellSignal, title="Sell Signal", location=location.abovebar, color=color.red, style=shape.triangledown, size=size.small)

// Divergence detection
 bullishDiv = ta.divergence(rsi, low, 5)
 bearishDiv = ta.divergence(rsi, high, 5)

plotshape(bullishDiv, title="Bullish Divergence", location=location.belowbar, color=color.lime, style=shape.triangleup, size=size.tiny)
plotshape(bearishDiv, title="Bearish Divergence", location=location.abovebar, color=color.red, style=shape.triangledown, size=size.tiny)
```

### How to Add to TradingView:
1. Open TradingView chart
2. Click on "Pine Editor" tab at the bottom
3. Copy and paste the script above
4. Click "Add to Chart"
5. Adjust parameters in the settings

### TradingView Built-in RSI:
1. Click on "Indicators" at the top
2. Search for "Relative Strength Index"
3. Select "RSI" from the list
4. Adjust length and overbought/oversold levels

## 8. Binance Integration

### How to Add RSI on Binance:
1. Open Binance trading interface
2. Select your trading pair (e.g., BTC/USDT)
3. Click on "Indicators" at the top of the chart
4. Search for "Relative Strength Index"
5. Select "RSI" from the list
6. In the settings:
   - Set "Length" (default: 14)
   - Adjust overbought/oversold levels
   - Choose color and line style
7. Click "Apply"

### Binance RSI Parameters:
- **Length**: 1-500 (common values: 9, 14, 21)
- **Source**: Close, Open, High, Low, HL/2, HLC/3, OHLC/4
- **Overbought**: Typically 70 or 80
- **Oversold**: Typically 30 or 20
- **Color**: Custom color selection
- **Line Style**: Solid, Dashed, Dotted
- **Line Width**: 1-5 pixels

## 9. Trading Strategies

### Strategy 1: RSI Overbought/Oversold
**Description**: Buy when RSI is oversold, sell when RSI is overbought

**Rules**:
- **Entry**: RSI crosses below oversold level (e.g., 30)
- **Exit**: RSI crosses above overbought level (e.g., 70)
- **Stop Loss**: 2-3% below entry price
- **Take Profit**: 5-10% above entry price or when opposite signal occurs

In [ ]:
def rsi_overbought_oversold_strategy(data, rsi_period=14, oversold=30, overbought=70):
    """
    RSI Overbought/Oversold Strategy
    """
    df = data.copy()
    
    # Calculate RSI
    df['rsi'] = IndicatorCalculator.rsi(df['close'], rsi_period)
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['rsi'] <= oversold, 'signal'] = 1  # Buy when oversold
    df.loc[df['rsi'] >= overbought, 'signal'] = -1  # Sell when overbought
    
    # Identify crossovers
    df['rsi_oversold_cross'] = ((df['rsi'] <= oversold) & 
                                (df['rsi'].shift(1) > oversold))
    df['rsi_overbought_cross'] = ((df['rsi'] >= overbought) & 
                                 (df['rsi'].shift(1) < overbought))
    
    return df

# Apply strategy
df_rsi_oo = rsi_overbought_oversold_strategy(df_manual)

# Plot signals
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'{pair} - RSI Overbought/Oversold Strategy', f'{pair} - RSI'),
    vertical_spacing=0.1,
    row_heights=[0.6, 0.4]
)

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_rsi_oo.index,
        open=df_rsi_oo['open'],
        high=df_rsi_oo['high'],
        low=df_rsi_oo['low'],
        close=df_rsi_oo['close'],
        name='Price'
    ),
    row=1, col=1
)

# Buy signals
buy_signals = df_rsi_oo[df_rsi_oo['rsi_oversold_cross']]
fig.add_trace(
    go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-up', size=12, color='green'),
        name='Buy Signal'
    ),
    row=1, col=1
)

# Sell signals
sell_signals = df_rsi_oo[df_rsi_oo['rsi_overbought_cross']]
fig.add_trace(
    go.Scatter(
        x=sell_signals.index,
        y=sell_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-down', size=12, color='red'),
        name='Sell Signal'
    ),
    row=1, col=1
)

# RSI chart
fig.add_trace(
    go.Scatter(
        x=df_rsi_oo.index,
        y=df_rsi_oo['rsi'],
        mode='lines',
        name='RSI(14)',
        line=dict(color='purple', width=2)
    ),
    row=2, col=1
)

# Overbought/Oversold levels
fig.add_hline(y=70, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_hline(y=30, line=dict(color='green', dash='dash'), row=2, col=1)

fig.update_layout(
    title=f'{pair} - RSI Overbought/Oversold Strategy',
    template='plotly_dark',
    height=800
)

fig.show()

print(f"Total buy signals: {len(buy_signals)}")
print(f"Total sell signals: {len(sell_signals)}")
print(f"\nRecent signals:")
recent_signals = df_rsi_oo[df_rsi_oo['rsi_oversold_cross'] | df_rsi_oo['rsi_overbought_cross']].tail(10)
display(recent_signals[['close', 'rsi', 'rsi_oversold_cross', 'rsi_overbought_cross']])

### Strategy 2: RSI Divergence
**Description**: Trade based on divergence between price and RSI

**Rules**:
- **Bullish Divergence**: Price makes lower lows while RSI makes higher lows
- **Bearish Divergence**: Price makes higher highs while RSI makes lower highs
- **Entry**: When divergence is confirmed
- **Exit**: When RSI reaches opposite extreme or divergence pattern completes

In [ ]:
def detect_rsi_divergence(data, rsi_period=14, lookback=5):
    """
    Detect RSI divergence patterns
    """
    df = data.copy()
    
    # Calculate RSI
    df['rsi'] = IndicatorCalculator.rsi(df['close'], rsi_period)
    
    # Find local minima and maxima
    df['price_low'] = df['close'].rolling(window=lookback, center=True).min()
    df['price_high'] = df['close'].rolling(window=lookback, center=True).max()
    df['rsi_low'] = df['rsi'].rolling(window=lookback, center=True).min()
    df['rsi_high'] = df['rsi'].rolling(window=lookback, center=True).max()
    
    # Detect divergences
    df['bullish_divergence'] = False
    df['bearish_divergence'] = False
    
    for i in range(lookback, len(df) - lookback):
        # Bullish divergence: lower price low, higher RSI low
        if (df['price_low'].iloc[i] < df['price_low'].iloc[i-lookback] and
            df['rsi_low'].iloc[i] > df['rsi_low'].iloc[i-lookback]):
            df.iloc[i, df.columns.get_loc('bullish_divergence')] = True
        
        # Bearish divergence: higher price high, lower RSI high
        if (df['price_high'].iloc[i] > df['price_high'].iloc[i-lookback] and
            df['rsi_high'].iloc[i] < df['rsi_high'].iloc[i-lookback]):
            df.iloc[i, df.columns.get_loc('bearish_divergence')] = True
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['bullish_divergence'], 'signal'] = 1
    df.loc[df['bearish_divergence'], 'signal'] = -1
    
    return df

# Apply strategy
df_rsi_div = detect_rsi_divergence(df_manual)

# Create signals DataFrame for backtesting
rsi_div_signals = pd.DataFrame(index=df_rsi_div.index)
rsi_div_signals['signal'] = df_rsi_div['signal']

print(f"Bullish divergences detected: {df_rsi_div['bullish_divergence'].sum()}")
print(f"Bearish divergences detected: {df_rsi_div['bearish_divergence'].sum()}")

# Plot divergences
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(df_rsi_div.index, df_rsi_div['close'], label='Price', alpha=0.7, color='black')
plt.scatter(df_rsi_div[df_rsi_div['bullish_divergence']].index, 
            df_rsi_div[df_rsi_div['bullish_divergence']]['close'], 
            color='green', s=100, marker='^', label='Bullish Divergence', zorder=5)
plt.scatter(df_rsi_div[df_rsi_div['bearish_divergence']].index, 
            df_rsi_div[df_rsi_div['bearish_divergence']]['close'], 
            color='red', s=100, marker='v', label='Bearish Divergence', zorder=5)
plt.title(f'{pair} - RSI Divergence Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_rsi_div.index, df_rsi_div['rsi'], label='RSI(14)', alpha=0.8, color='purple')
plt.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold (30)')
plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

### Strategy 3: RSI with Trend Filter
**Description**: Use RSI signals only in the direction of the main trend

**Rules**:
- **Uptrend**: Price above 200-day SMA, only take RSI buy signals
- **Downtrend**: Price below 200-day SMA, only take RSI sell signals
- **Entry**: RSI overbought/oversold in trend direction
- **Exit**: RSI reaches opposite extreme or trend changes

In [ ]:
def rsi_trend_filter_strategy(data, rsi_period=14, trend_period=200, oversold=30, overbought=70):
    """
    RSI with Trend Filter Strategy
    """
    df = data.copy()
    
    # Calculate indicators
    df['rsi'] = IndicatorCalculator.rsi(df['close'], rsi_period)
    df['trend_sma'] = IndicatorCalculator.sma(df['close'], trend_period)
    
    # Determine trend
    df['uptrend'] = df['close'] > df['trend_sma']
    df['downtrend'] = df['close'] < df['trend_sma']
    
    # Generate signals with trend filter
    df['signal'] = 0
    
    # Buy signals: RSI oversold AND uptrend
    buy_condition = (df['rsi'] <= oversold) & df['uptrend']
    
    # Sell signals: RSI overbought AND downtrend
    sell_condition = (df['rsi'] >= overbought) & df['downtrend']
    
    df.loc[buy_condition, 'signal'] = 1
    df.loc[sell_condition, 'signal'] = -1
    
    return df

# Apply strategy
df_rsi_trend = rsi_trend_filter_strategy(df_manual)

# Create signals DataFrame for backtesting
rsi_trend_signals = pd.DataFrame(index=df_rsi_trend.index)
rsi_trend_signals['signal'] = df_rsi_trend['signal']

print(f"Buy signals (oversold + uptrend): {(df_rsi_trend['signal'] == 1).sum()}")
print(f"Sell signals (overbought + downtrend): {(df_rsi_trend['signal'] == -1).sum()}")

# Plot strategy
plt.figure(figsize=(15, 12))

plt.subplot(3, 1, 1)
plt.plot(df_rsi_trend.index, df_rsi_trend['close'], label='Price', alpha=0.7, color='black')
plt.plot(df_rsi_trend.index, df_rsi_trend['trend_sma'], label='SMA(200)', alpha=0.8, color='blue')
plt.scatter(df_rsi_trend[df_rsi_trend['signal'] == 1].index, 
            df_rsi_trend[df_rsi_trend['signal'] == 1]['close'], 
            color='green', s=100, marker='^', label='Buy Signal', zorder=5)
plt.scatter(df_rsi_trend[df_rsi_trend['signal'] == -1].index, 
            df_rsi_trend[df_rsi_trend['signal'] == -1]['close'], 
            color='red', s=100, marker='v', label='Sell Signal', zorder=5)
plt.title(f'{pair} - RSI with Trend Filter Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(df_rsi_trend.index, df_rsi_trend['rsi'], label='RSI(14)', alpha=0.8, color='purple')
plt.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold (30)')
plt.title('Relative Strength Index (RSI)')
plt.ylabel('RSI')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.subplot(3, 1, 3)
plt.plot(df_rsi_trend.index, df_rsi_trend['uptrend'], label='Uptrend', alpha=0.7, color='green')
plt.plot(df_rsi_trend.index, df_rsi_trend['downtrend'], label='Downtrend', alpha=0.7, color='red')
plt.title('Trend Filter')
plt.xlabel('Date')
plt.ylabel('Trend')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

## 10. Backtesting Results

Let's backtest our strategies and analyze the results.

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(initial_capital=10000)

# Create signals for each strategy
strategies = {
    'RSI_Overbought_Oversold': df_rsi_oo[['signal']],
    'RSI_Divergence': rsi_div_signals,
    'RSI_Trend_Filter': rsi_trend_signals
}

# Run backtests
for name, signals in strategies.items():
    print(f"Backtesting {name} strategy...")
    engine.run_backtest(df_manual, signals, name)

# Compare strategies
comparison = engine.compare_strategies(list(strategies.keys()))
print("\nStrategy Comparison:")
display(comparison)

# Plot equity curves
engine.plot_equity_curve()

# Plot drawdowns
engine.plot_drawdown()

In [ ]:
# Generate detailed report for the best strategy
best_strategy = comparison['total_return'].idxmax()
print(f"Best performing strategy: {best_strategy}")
print(engine.generate_report(best_strategy))

In [ ]:
# Analyze trade statistics for the best strategy
best_data = engine.results[best_strategy]['data']

# Analyze RSI levels at trade entries
trade_analysis = best_data[best_data['signal'] != 0].copy()
trade_analysis['rsi'] = df_manual['rsi_14']

if len(trade_analysis) > 0:
    print(f"\nTrade Analysis for {best_strategy}:")
    print(f"Total trades: {len(trade_analysis)}")
    print(f"Buy trades: {len(trade_analysis[trade_analysis['signal'] == 1])}")
    print(f"Sell trades: {len(trade_analysis[trade_analysis['signal'] == -1])}")
    print(f"Average RSI at entry: {trade_analysis['rsi'].mean():.1f}")
    print(f"RSI range at entry: {trade_analysis['rsi'].min():.1f} - {trade_analysis['rsi'].max():.1f}")
    
    # RSI distribution at trade entries
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(trade_analysis['rsi'], bins=20, alpha=0.7, edgecolor='black')
    plt.axvline(x=30, color='green', linestyle='--', label='Oversold (30)')
    plt.axvline(x=70, color='red', linestyle='--', label='Overbought (70)')
    plt.axvline(x=50, color='gray', linestyle=':', label='Midline (50)')
    plt.title('RSI Distribution at Trade Entries')
    plt.xlabel('RSI')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Performance by RSI zone
    trade_analysis['rsi_zone'] = pd.cut(trade_analysis['rsi'], 
                                      bins=[0, 30, 50, 70, 100], 
                                      labels=['Oversold', 'Low', 'High', 'Overbought'])
    
    plt.subplot(1, 2, 2)
    zone_performance = trade_analysis.groupby('rsi_zone')['strategy_returns'].mean() * 100
    zone_performance.plot(kind='bar', color=['green', 'blue', 'orange', 'red'])
    plt.title('Average Returns by RSI Zone')
    plt.xlabel('RSI Zone')
    plt.ylabel('Average Return (%)')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades completed in the backtest period")

## 11. Pros and Cons

### Advantages:
- ✅ **Leading Indicator**: Can signal trend changes before they occur
- ✅ **Overbought/Oversold**: Clear levels for extreme conditions
- ✅ **Divergence**: Early warning of potential reversals
- ✅ **Versatile**: Works in various market conditions
- ✅ **Standardized**: Consistent interpretation across markets

### Disadvantages:
- ❌ **False Signals**: Can remain overbought/oversold for extended periods
- ❌ **Trend Strength**: Doesn't indicate the strength of trends
- ❌ **Whipsaw**: Can generate false signals in choppy markets
- ❌ **Lagging**: Based on historical price data

### When to Use:
- **Ranging Markets**: Excellent for sideways price action
- **Reversal Points**: Good for identifying potential turning points
- **Confirmation**: To confirm other indicators or signals
- **Multiple Timeframes**: For confluence analysis

### When to Avoid:
- **Strong Trends**: Can give false signals in trending markets
- **Low Volatility**: Less reliable in low volatility periods
- **As Standalone**: Should not be used as the only indicator
- **News Events**: Can be unreliable during high volatility

## 12. Advanced RSI Techniques

### 1. RSI with Multiple Timeframes
Use RSI on different timeframes to get a broader market perspective:
- **Long-term**: RSI(21) on daily charts for overall trend
- **Medium-term**: RSI(14) on 4-hour charts for intermediate signals
- **Short-term**: RSI(9) on 1-hour charts for entry timing

### 2. RSI and Moving Averages
Combine RSI with moving averages for better signals:
- **RSI MA Cross**: Signal when RSI crosses its own moving average
- **Trend Filter**: Only trade RSI signals in the direction of price MA

### 3. RSI Price Bands
Create dynamic overbought/oversold levels based on volatility:
- **Volatility Adjusted**: Adjust levels based on ATR
- **Regime Detection**: Different levels for trending vs. ranging markets

In [ ]:
# Advanced RSI Techniques
df_advanced = df_manual.copy()

# 1. RSI with Moving Averages
df_advanced['rsi_ma_9'] = df_advanced['rsi_14'].rolling(window=9).mean()
df_advanced['rsi_ma_21'] = df_advanced['rsi_14'].rolling(window=21).mean()

# 2. RSI Volatility Bands
df_advanced['rsi_std'] = df_advanced['rsi_14'].rolling(window=14).std()
df_advanced['rsi_upper_band'] = df_advanced['rsi_14'] + (df_advanced['rsi_std'] * 0.5)
df_advanced['rsi_lower_band'] = df_advanced['rsi_14'] - (df_advanced['rsi_std'] * 0.5)

# 3. RSI Rate of Change
df_advanced['rsi_roc'] = df_advanced['rsi_14'].pct_change() * 100

# 4. RSI and Price Correlation
df_advanced['price_rsi_corr'] = df_advanced['close'].rolling(window=20).corr(df_advanced['rsi_14'])

# Plot advanced techniques
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# RSI with Moving Averages
axes[0, 0].plot(df_advanced.index, df_advanced['rsi_14'], label='RSI(14)', color='purple')
axes[0, 0].plot(df_advanced.index, df_advanced['rsi_ma_9'], label='RSI MA(9)', color='orange')
axes[0, 0].plot(df_advanced.index, df_advanced['rsi_ma_21'], label='RSI MA(21)', color='blue')
axes[0, 0].axhline(y=70, color='red', linestyle='--', alpha=0.5)
axes[0, 0].axhline(y=30, color='green', linestyle='--', alpha=0.5)
axes[0, 0].set_title('RSI with Moving Averages')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylim(0, 100)

# RSI Volatility Bands
axes[0, 1].plot(df_advanced.index, df_advanced['rsi_14'], label='RSI(14)', color='purple')
axes[0, 1].plot(df_advanced.index, df_advanced['rsi_upper_band'], label='Upper Band', color='red', linestyle='--')
axes[0, 1].plot(df_advanced.index, df_advanced['rsi_lower_band'], label='Lower Band', color='green', linestyle='--')
axes[0, 1].fill_between(df_advanced.index, df_advanced['rsi_lower_band'], 
                        df_advanced['rsi_upper_band'], alpha=0.1, color='gray')
axes[0, 1].set_title('RSI Volatility Bands')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim(0, 100)

# RSI Rate of Change
axes[1, 0].plot(df_advanced.index, df_advanced['rsi_roc'], label='RSI Rate of Change', color='orange')
axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 0].fill_between(df_advanced.index, df_advanced['rsi_roc'], 0, alpha=0.3, color='orange')
axes[1, 0].set_title('RSI Rate of Change')
axes[1, 0].set_ylabel('Rate of Change (%)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# RSI and Price Correlation
axes[1, 1].plot(df_advanced.index, df_advanced['price_rsi_corr'], label='Price-RSI Correlation', color='blue')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Positive Correlation')
axes[1, 1].axhline(y=-0.5, color='red', linestyle='--', alpha=0.5, label='Negative Correlation')
axes[1, 1].set_title('Price-RSI Correlation')
axes[1, 1].set_ylabel('Correlation')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(-1, 1)

plt.tight_layout()
plt.show()

# Analysis of extreme RSI movements
print("Extreme RSI Movement Analysis:")
extreme_roc = df_advanced[abs(df_advanced['rsi_roc']) > 20][['close', 'rsi_14', 'rsi_roc']]

if len(extreme_roc) > 0:
    print(f"Found {len(extreme_roc)} instances of extreme RSI movements (>20% in one period)")
    display(extreme_roc.head())
else:
    print("No extreme RSI movements found in the dataset")

## 13. Risk Management with RSI

### 1. Position Sizing Based on RSI Levels
- **Extreme Levels**: Larger positions when RSI is at extremes
- **Moderate Levels**: Smaller positions when RSI is near 50
- **Divergence**: Increased position size on divergence signals

### 2. Stop Loss Placement
- **Oversold Buys**: Place stop loss below recent swing low
- **Overbought Sells**: Place stop loss above recent swing high
- **ATR-Based**: Use ATR to determine stop loss distance

### 3. Take Profit Levels
- **RSI Targets**: Take profit when RSI reaches opposite extreme
- **Scaling Out**: Partial profits at different RSI levels
- **Trailing Stops**: Use RSI levels for trailing stop adjustments

In [ ]:
# Risk Management with RSI
df_risk = df_manual.copy()

# Calculate ATR for volatility-based stops
df_risk['atr'] = IndicatorCalculator.atr(df_risk['high'], df_risk['low'], df_risk['close'], 14)

# Risk management techniques
df_risk['rsi_14'] = IndicatorCalculator.rsi(df_risk['close'], 14)

# 1. Position sizing based on RSI levels
df_risk['rsi_distance_from_center'] = abs(df_risk['rsi_14'] - 50)
df_risk['position_size'] = np.where(
    df_risk['rsi_distance_from_center'] > 20, 1.0,  # Large position at extremes
    np.where(
        df_risk['rsi_distance_from_center'] > 10, 0.5,  # Medium position
        0.25  # Small position near center
    )
)

# 2. Stop loss based on RSI levels and ATR
stop_loss_multiplier = 1.5
df_risk['stop_loss_long'] = df_risk['close'] - (stop_loss_multiplier * df_risk['atr'])
df_risk['stop_loss_short'] = df_risk['close'] + (stop_loss_multiplier * df_risk['atr'])

# 3. Risk-reward ratio based on RSI distance
df_risk['rsi_target_distance'] = abs(df_risk['rsi_14'] - 50)
df_risk['reward_per_unit'] = df_risk['rsi_target_distance'] * 0.5  # Scale factor
df_risk['risk_per_unit'] = df_risk['atr'] * stop_loss_multiplier
df_risk['risk_reward_ratio'] = df_risk['reward_per_unit'] / df_risk['risk_per_unit']

# Plot risk management
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position sizing
axes[0, 0].plot(df_risk.index, df_risk['position_size'], label='Position Size', color='purple', linewidth=2)
axes[0, 0].fill_between(df_risk.index, df_risk['position_size'], alpha=0.3, color='purple')
axes[0, 0].set_title('Position Sizing Based on RSI Distance from Center')
axes[0, 0].set_ylabel('Position Size (Multiplier)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# RSI levels and position size
axes[0, 1].scatter(df_risk['rsi_14'], df_risk['position_size'], alpha=0.6, c=df_risk['rsi_14'], cmap='RdYlGn')
axes[0, 1].axvline(x=30, color='green', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=70, color='red', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=50, color='gray', linestyle=':', alpha=0.5)
axes[0, 1].set_title('Position Size vs RSI Level')
axes[0, 1].set_xlabel('RSI')
axes[0, 1].set_ylabel('Position Size')
axes[0, 1].grid(True, alpha=0.3)

# Risk-reward ratio
axes[1, 0].plot(df_risk.index, df_risk['risk_reward_ratio'], label='Risk-Reward Ratio', color='orange')
axes[1, 0].axhline(y=2, color='red', linestyle='--', label='Minimum 2:1 Ratio')
axes[1, 0].fill_between(df_risk.index, df_risk['risk_reward_ratio'], 2, alpha=0.3, color='orange')
axes[1, 0].set_title('Risk-Reward Ratio Analysis')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# RSI distance distribution
axes[1, 1].hist(df_risk['rsi_distance_from_center'].dropna(), bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[1, 1].axvline(x=df_risk['rsi_distance_from_center'].quantile(0.7), color='red', linestyle='--', label='70th percentile')
axes[1, 1].axvline(x=df_risk['rsi_distance_from_center'].quantile(0.3), color='green', linestyle='--', label='30th percentile')
axes[1, 1].set_title('RSI Distance from Center Distribution')
axes[1, 1].set_xlabel('Distance from 50')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risk management statistics
print("Risk Management Statistics:")
print(f"Average position size: {df_risk['position_size'].mean():.2f}")
print(f"Average risk-reward ratio: {df_risk['risk_reward_ratio'].mean():.2f}")
print(f"Trades with minimum 2:1 ratio: {(df_risk['risk_reward_ratio'] >= 2).sum()} ({(df_risk['risk_reward_ratio'] >= 2).mean()*100:.1f}%)")
print(f"Average RSI distance from center: {df_risk['rsi_distance_from_center'].mean():.1f}")

## 14. Conclusion

The Relative Strength Index (RSI) is a powerful momentum oscillator that provides valuable insights into market conditions. While it has limitations, particularly in trending markets, it remains one of the most reliable indicators for identifying overbought/oversold conditions and potential reversals.

### Key Takeaways:
1. **Momentum Measurement**: RSI excels at measuring the speed of price movements
2. **Extreme Conditions**: Excellent for identifying overbought/oversold levels
3. **Divergence Detection**: Early warning of potential trend reversals
4. **Trend Confirmation**: Can be used to confirm the strength of existing trends
5. **Combination**: Works best when combined with trend indicators and volume analysis

### Best Practices:
- Use multiple RSI periods for confirmation
- Combine with trend indicators like moving averages
- Implement proper risk management techniques
- Consider market conditions when interpreting RSI signals
- Test different parameters to find what works best for your trading style

## 15. Exercises

### Exercise 1: Optimize RSI Parameters
Test different RSI periods and overbought/oversold levels to find the optimal combination for different market conditions. Create a function that tests multiple parameter combinations and returns the best performing one based on Sharpe ratio.

### Exercise 2: Create RSI Strategy with Volume Confirmation
Modify the RSI overbought/oversold strategy to include volume confirmation. Only enter trades when volume is above the 20-day average and RSI signals are confirmed.

### Exercise 3: Implement RSI with Multiple Timeframe Analysis
Create a strategy that uses RSI signals from multiple timeframes (e.g., daily, 4-hour, 1-hour) and only enters trades when signals align across timeframes.

### Solutions:
```python
# Exercise 1 Solution
def optimize_rsi_parameters(data, rsi_range, ob_range, os_range):
    best_sharpe = -float('inf')
    best_params = None
    
    for rsi_period in rsi_range:
        for ob_level in ob_range:
            for os_level in os_range:
                if ob_level <= os_level:
                    continue
                
                signals = rsi_overbought_oversold_strategy(data, rsi_period, os_level, ob_level)[['signal']]
                engine = BacktestEngine()
                result = engine.run_backtest(data, signals, f"RSI_{rsi_period}_{ob_level}_{os_level}")
                sharpe = result['metrics']['sharpe_ratio']
                
                if sharpe > best_sharpe:
                    best_sharpe = sharpe
                    best_params = (rsi_period, ob_level, os_level)
    
    return best_params, best_sharpe

# Exercise 2 Solution
def rsi_volume_strategy(data, rsi_period=14, oversold=30, overbought=70, volume_period=20):
    df = data.copy()
    
    # Calculate RSI and volume SMA
    df['rsi'] = IndicatorCalculator.rsi(df['close'], rsi_period)
    df['volume_sma'] = IndicatorCalculator.sma(df['volume'], volume_period)
    
    # Generate signals with volume confirmation
    df['signal'] = 0
    
    # Buy signal: RSI oversold AND high volume
    buy_condition = ((df['rsi'] <= oversold) & 
                    (df['volume'] > df['volume_sma']))
    
    # Sell signal: RSI overbought AND high volume
    sell_condition = ((df['rsi'] >= overbought) & 
                      (df['volume'] > df['volume_sma']))
    
    df.loc[buy_condition, 'signal'] = 1
    df.loc[sell_condition, 'signal'] = -1
    
    return df
```

## 16. Additional Resources

- **Books**:
  - "New Concepts in Technical Trading Systems" by J. Welles Wilder Jr.
  - "Technical Analysis of Financial Markets" by John J. Murphy
  - "Trading Systems and Methods" by Perry J. Kaufman

- **Online Resources**:
  - [TradingView Documentation](https://www.tradingview.com/pine-script-docs/)
  - [Investopedia - RSI](https://www.investopedia.com/terms/r/rsi.asp)
  - [Binance Academy - Technical Analysis](https://academy.binance.com/en/categories/technical-analysis)

- **Research Papers**:
  - "The Profitability of Technical Analysis in the Foreign Exchange Market" by Cheol-Ho Park and Scott H. Irwin
  - "Technical Analysis and Momentum Strategies in Commodity Futures" by Szakmary and Shen

- **Tools**:
  - [TradingView](https://www.tradingview.com/) - Charting and Pine Script
  - [Binance](https://www.binance.com/) - Trading and indicators
  - [Freqtrade](https://www.freqtrade.io/) - Algorithmic trading

---

**Disclaimer**: This tutorial is for educational purposes only. Trading cryptocurrencies involves significant risk. Always do your own research and consider consulting with a financial advisor before making investment decisions.